In [5]:
#Loading the devtest files of Flores-200 datasets

import os
import pandas as pd

def load_devtest_files(devtest_directory, specific_files):
    all_data = {}

    for file in specific_files:
        file_path = os.path.join(devtest_directory, file)
        
        if os.path.exists(file_path):
            try:
 
                with open(file_path, 'r', encoding='utf-8') as f:
                    sentences = f.readlines()
                sentences = [sentence.strip() for sentence in sentences if sentence.strip()] 

                all_data[file] = sentences
                print(f"Loaded: {file} with {len(sentences)} sentences")
            except Exception as e:
                print(f"Error loading {file}: {e}")
        else:
            print(f"File not found: {file}")

    return all_data


devtest_directory = r'D:\Machine translation model\flores200_dataset\flores200_dataset\devtest'
specific_files = [
    'eng_Latn.devtest',
    'hin_Deva.devtest',
    'mar_Deva.devtest',
    'guj_Gujr.devtest',
    'ben_Beng.devtest',
    'tel_Telu.devtest',
    'tam_Taml.devtest',
]


loaded_data = load_devtest_files(devtest_directory, specific_files)

Loaded: eng_Latn.devtest with 1012 sentences
Loaded: hin_Deva.devtest with 1012 sentences
Loaded: mar_Deva.devtest with 1012 sentences
Loaded: guj_Gujr.devtest with 1012 sentences
Loaded: ben_Beng.devtest with 1012 sentences
Loaded: tel_Telu.devtest with 1012 sentences
Loaded: tam_Taml.devtest with 1012 sentences


In [7]:
#Machine translation from English to 5 Indian languages['hin', 'mar', 'guj', 'ben', 'tel', 'tam'] of 150 random samples
#Saving the data into excel files by parallel_dataset_eng_lang

def save_random_samples_to_excel(loaded_data, sample_size=150):
    english_sentences = loaded_data['eng_Latn.devtest']
    
    output_directory = r'D:\Machine translation model\Machine Translation Model\OUTPUT MT'

    sample_size = min(sample_size, len(english_sentences))

    for lang_file, lang_sentences in loaded_data.items():
        if lang_file != 'eng_Latn.devtest':
    
            min_len = min(len(english_sentences), len(lang_sentences))
            if min_len < sample_size:
                print(f"Not enough sentences to sample for {lang_file}. Available: {min_len}")
                continue
            
            english_sample = pd.Series(english_sentences).sample(n=sample_size, random_state=42).reset_index(drop=True)
            lang_sample = pd.Series(lang_sentences).sample(n=sample_size, random_state=42).reset_index(drop=True)

            parallel_df = pd.DataFrame({
                'English': english_sample,
                'Translation': lang_sample
            })

            lang_code = lang_file.split('_')[0]
            excel_filename = os.path.join(output_directory, f'parallel_dataset_eng_{lang_code}.xlsx')
            parallel_df.to_excel(excel_filename, index=False)
            print(f'Saved: {excel_filename} with {len(parallel_df)} entries.')
save_random_samples_to_excel(loaded_data)


Saved: D:\Machine translation model\Machine Translation Model\OUTPUT MT\parallel_dataset_eng_hin.xlsx with 150 entries.
Saved: D:\Machine translation model\Machine Translation Model\OUTPUT MT\parallel_dataset_eng_mar.xlsx with 150 entries.
Saved: D:\Machine translation model\Machine Translation Model\OUTPUT MT\parallel_dataset_eng_guj.xlsx with 150 entries.
Saved: D:\Machine translation model\Machine Translation Model\OUTPUT MT\parallel_dataset_eng_ben.xlsx with 150 entries.
Saved: D:\Machine translation model\Machine Translation Model\OUTPUT MT\parallel_dataset_eng_tel.xlsx with 150 entries.
Saved: D:\Machine translation model\Machine Translation Model\OUTPUT MT\parallel_dataset_eng_tam.xlsx with 150 entries.


In [9]:
#loading the saved excel file of Machine translation

import pandas as pd
import os

def load_parallel_samples(output_directory, languages):
    sampled_data = {}
    for lang_code in languages:
        excel_filename = os.path.join(output_directory, f'parallel_dataset_eng_{lang_code}.xlsx')
        if os.path.exists(excel_filename):
            df = pd.read_excel(excel_filename)
            sampled_data[lang_code] = df
            print(f"Loaded: {excel_filename} with {len(df)} entries.")
        else:
            print(f"File not found: {excel_filename}")
    return sampled_data

output_directory = r'D:\Machine translation model\Machine Translation Model\OUTPUT MT'
languages = ['hin', 'mar', 'guj', 'ben', 'tel', 'tam'] 
sampled_data = load_parallel_samples(output_directory, languages)

Loaded: D:\Machine translation model\Machine Translation Model\OUTPUT MT\parallel_dataset_eng_hin.xlsx with 150 entries.
Loaded: D:\Machine translation model\Machine Translation Model\OUTPUT MT\parallel_dataset_eng_mar.xlsx with 150 entries.
Loaded: D:\Machine translation model\Machine Translation Model\OUTPUT MT\parallel_dataset_eng_guj.xlsx with 150 entries.
Loaded: D:\Machine translation model\Machine Translation Model\OUTPUT MT\parallel_dataset_eng_ben.xlsx with 150 entries.
Loaded: D:\Machine translation model\Machine Translation Model\OUTPUT MT\parallel_dataset_eng_tel.xlsx with 150 entries.
Loaded: D:\Machine translation model\Machine Translation Model\OUTPUT MT\parallel_dataset_eng_tam.xlsx with 150 entries.


In [13]:
import cohere
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import pandas as pd
import os
import time

# Replace with your actual Cohere API key
cohere_api_key = 'API KEYS'
co = cohere.Client(cohere_api_key)

def evaluate_translation(english_sentence, actual_translation, target_language):
    prompt = (
        f"Translate the following English sentence into **{target_language} only**, ensuring the translation is natural and fluent. "
        "Please only use one of these languages: Hindi, Marathi, Bengali, Telugu, Gujarati, or Tamil. "
        "Avoid mixing any other languages or dialects in the translation. "
        "Avoid Having Bleu score = 0 and regenerate the translation"
        "Make sure that the translation is grammatically correct and contextually appropriate.\n\n"
        f"English sentence: {english_sentence}\n"
        f"Translation in {target_language}:"
    )

    try:
        response = co.generate(
            model='command-r-plus',
            prompt=prompt,
            max_tokens=500,  
            temperature=0.3,
        )
        cohere_translation = response.generations[0].text.strip()  # No cleaning of the translation
        if cohere_translation:
            # Apply smoothing for BLEU score calculation
            smoothie = SmoothingFunction().method4
            bleu_score = sentence_bleu([actual_translation.split()], cohere_translation.split(), smoothing_function=smoothie)
        else:
            bleu_score = 0
        
        return bleu_score, cohere_translation

    except Exception as e:
        print(f"Error evaluating translation for sentence '{english_sentence}': {e}")
        return 0.0, ""

def evaluate_samples_and_save_to_excel(sampled_data, output_directory):
    for lang_code, data in sampled_data.items():
        target_language = lang_code.split('_')[0]
        english_sentences = data['English']
        actual_translations = data['Translation']

        bleu_scores = []
        cohere_translations = []

        for english_sentence, actual_translation in zip(english_sentences, actual_translations):
            bleu_score, cohere_translation = evaluate_translation(english_sentence, actual_translation, target_language)
            bleu_scores.append(bleu_score)
            cohere_translations.append(cohere_translation)

            print(f"Evaluated: {english_sentence} -> {cohere_translation} | BLEU Score: {bleu_score}")
            time.sleep(1)  # Rate limit handling

        evaluation_df = pd.DataFrame({
            'English': english_sentences,
            'Actual_Translation': actual_translations,
            'Cohere_Translation': cohere_translations,
            'BLEU_Score': bleu_scores
        })

        excel_filename = os.path.join(output_directory, f'evaluation_eng_{lang_code}proved.xlsx')
        # Use the 'xlsxwriter' engine to ensure UTF-8-SIG encoding
        with pd.ExcelWriter(excel_filename, engine='xlsxwriter') as writer:
            evaluation_df.to_excel(writer, index=False, sheet_name='Evaluation')

        print(f"Saved evaluation results to: {excel_filename}")

# Assuming sampled_data and output_directory are defined
evaluate_samples_and_save_to_excel(sampled_data, output_directory)


Evaluated: Vatican City's population is around 800. It is the smallest independent country in the world and the country with the lowest population. -> वेटिकन सिटी की आबादी लगभग 800 है। यह दुनिया का सबसे छोटा स्वतंत्र देश है और आबादी के हिसाब से सबसे छोटा देश भी है। | BLEU Score: 0.10876623533099877
Evaluated: All citizens of Vatican City are Roman Catholic. -> वेटिकन सिटी के सभी नागरिक रोमन कैथोलिक हैं। | BLEU Score: 0.8408964152537145
Evaluated: It has a notably wide variety of plant communities, due to its range of microclimates, differing soils and varying levels of altitude. -> यहाँ माइक्रोक्लाइमेट, अलग-अलग मिट्टी और बदलती ऊंचाई के कारण पाए जाने वाले पौधों की विविधता प्रमुख रूप से व्यापक है। | BLEU Score: 0.039008168236700266
Evaluated: The Amazon River is the second longest and the biggest river on Earth. It carries more than 8 times as much water as the second biggest river. -> अमेज़न नदी पृथ्वी पर दूसरी सबसे लंबी और सबसे बड़ी नदी है। यह दूसरी सबसे बड़ी नदी से आठ गुना अधिक पानी ल

In [1]:
import pandas as pd
import os

def load_specific_excel_files(input_directory, specific_files):
    loaded_data = {}

    for specific_file in specific_files:
        excel_path = os.path.join(input_directory, f'{specific_file}.xlsx')
        
        if os.path.exists(excel_path):
            try:
                df = pd.read_excel(excel_path)
                loaded_data[specific_file] = df
                print(f"Loaded data from: {excel_path}")
            except Exception as e:
                print(f"Error loading {excel_path}: {e}")
        else:
            print(f"File not found: {excel_path}")

    return loaded_data

input_directory = 'D:\Machine translation model\Machine Translation Model\OUTPUT EVALUATION\Cohere eval'  
specific_files = ['evaluation_eng_hin-cohere', 'evaluation_eng_ben-cohere', 'evaluation_eng_guj-cohere','evaluation_eng_mar-cohere','evaluation_eng_tam-cohere','evaluation_eng_tel-cohere']  
loaded_data = load_specific_excel_files(input_directory, specific_files)

Loaded data from: D:\Machine translation model\Machine Translation Model\OUTPUT EVALUATION\Cohere eval\evaluation_eng_hin-cohere.xlsx
Loaded data from: D:\Machine translation model\Machine Translation Model\OUTPUT EVALUATION\Cohere eval\evaluation_eng_ben-cohere.xlsx
Loaded data from: D:\Machine translation model\Machine Translation Model\OUTPUT EVALUATION\Cohere eval\evaluation_eng_guj-cohere.xlsx
Loaded data from: D:\Machine translation model\Machine Translation Model\OUTPUT EVALUATION\Cohere eval\evaluation_eng_mar-cohere.xlsx
Loaded data from: D:\Machine translation model\Machine Translation Model\OUTPUT EVALUATION\Cohere eval\evaluation_eng_tam-cohere.xlsx
Loaded data from: D:\Machine translation model\Machine Translation Model\OUTPUT EVALUATION\Cohere eval\evaluation_eng_tel-cohere.xlsx


In [27]:
import pandas as pd
import cohere
import random
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# Initialize Cohere client with your API key
cohere_api_key = 'API KEYS'
co = cohere.Client(cohere_api_key)

def regenerate_translation(english_sentence, target_language):
    try:
        # Call the Cohere API for translation
        response = co.generate(
            model='command-r-plus',
            prompt= f"Translate the following English sentence into **{target_language} only**, ensuring the translation is natural and fluent. "
                    "Please only use one of these languages: Hindi, Marathi, Bengali, Telugu, Gujarati, or Tamil. "
                    "Avoid mixing any other languages or dialects in the translation. "
                    "Avoid Having Bleu score = 0 and regenerate the translation"
                    "Make sure that the translation is grammatically correct and contextually appropriate.\n\n"
                    f"English sentence: {english_sentence}\n"
                    f"Translation in {target_language}:",
            max_tokens=500,
            temperature=random.uniform(0.2, 0.3),
            stop_sequences=["."],
        )
        # Extract the translation from the response
        if response and response.generations:
            return response.generations[0].text.strip()
        else:
            print("No translation generated.")
            return None
    except Exception as e:
        print(f"Error during translation generation: {e}")
        return None

def calculate_bleu_score(actual_translation, cohere_translation):
    reference = actual_translation.split()  # Reference translation
    candidate = cohere_translation.split()  # Generated translation
    
    # Use smoothing function for BLEU score calculation
    smoothie = SmoothingFunction().method4
    return sentence_bleu([reference], candidate, smoothing_function=smoothie)

def update_translations_with_zero_bleu(input_file, output_file):
    df = pd.read_excel(input_file)
    print("Columns in the Excel file:", df.columns)

    required_columns = ['English', 'Actual_Translation', 'Cohere_Translation', 'BLEU_Score']
    for col in required_columns:
        if col not in df.columns:
            print(f"Error: Required column '{col}' not found in the input file.")
            return

    zero_bleu_rows = df[df['BLEU_Score'] == 0]

    if zero_bleu_rows.empty:
        print("No rows with a BLEU score of 0 found.")
        return

    if 'evaluation_eng_hin-cohere' in input_file:
        target_language = 'Hindi'
    elif 'evaluation_eng_ben-cohere' in input_file:
        target_language = 'Bengali'
    elif 'evaluation_eng_guj-cohere' in input_file:
        target_language = 'Gujarati'
    elif 'evaluation_eng_mar-cohere' in input_file:
        target_language = 'Marathi'
    elif 'evaluation_eng_tam-cohere' in input_file:
        target_language = 'Tamil'
    elif 'evaluation_eng_tel-cohere' in input_file:
        target_language = 'Telugu'
    else:
        print("Error: Unrecognized input file for setting target language.")
        return

    for index, row in zero_bleu_rows.iterrows():
        english_sentence = row['English']
        actual_translation = row['Actual_Translation']

        # Regenerate the translation
        cohere_translation = regenerate_translation(english_sentence, target_language)

        # Calculate the new BLEU score
        if cohere_translation:
            new_bleu_score = calculate_bleu_score(actual_translation, cohere_translation)

            # Update the DataFrame
            df.at[index, 'Cohere_Translation'] = cohere_translation
            df.at[index, 'BLEU_Score'] = new_bleu_score

            print(f"Regenerated: {english_sentence} -> {cohere_translation} | New BLEU Score: {new_bleu_score}")
        else:
            print(f"Failed to regenerate translation for: {english_sentence}")

    # Save the updated DataFrame back to an Excel file
    df.to_excel(output_file, index=False)
    print(f"Updated data saved to: {output_file}")

# Example usage for multiple files
files = [
    {
        'input_file': r'D:\Machine translation model\Machine Translation Model\OUTPUT EVALUATION\Cohere eval\evaluation_eng_hin-cohere.xlsx',
        'output_file': r'D:\Machine translation model\Machine Translation Model\OUTPUT EVALUATION\Cohere eval\evaluation_eng_hin-cohereupdated.xlsx'
    },
    {
        'input_file': r'D:\Machine translation model\Machine Translation Model\OUTPUT EVALUATION\Cohere eval\evaluation_eng_ben-cohere.xlsx',
        'output_file': r'D:\Machine translation model\Machine Translation Model\OUTPUT EVALUATION\Cohere eval\evaluation_eng_ben-cohereupdated.xlsx'
    },
    {
        'input_file': r'D:\Machine translation model\Machine Translation Model\OUTPUT EVALUATION\Cohere eval\evaluation_eng_guj-cohere.xlsx',
        'output_file': r'D:\Machine translation model\Machine Translation Model\OUTPUT EVALUATION\Cohere eval\evaluation_eng_guj-cohereupdated.xlsx'
    },
    {
        'input_file': r'D:\Machine translation model\Machine Translation Model\OUTPUT EVALUATION\Cohere eval\evaluation_eng_mar-cohere.xlsx',
        'output_file': r'D:\Machine translation model\Machine Translation Model\OUTPUT EVALUATION\Cohere eval\evaluation_eng_mar-cohereupdated.xlsx'
    },
    {
        'input_file': r'D:\Machine translation model\Machine Translation Model\OUTPUT EVALUATION\Cohere eval\evaluation_eng_tam-cohere.xlsx',
        'output_file': r'D:\Machine translation model\Machine Translation Model\OUTPUT EVALUATION\Cohere eval\evaluation_eng_tam-cohereupdated.xlsx'
    },
    {
        'input_file': r'D:\Machine translation model\Machine Translation Model\OUTPUT EVALUATION\Cohere eval\evaluation_eng_tel-cohere.xlsx',
        'output_file': r'D:\Machine translation model\Machine Translation Model\OUTPUT EVALUATION\Cohere eval\evaluation_eng_tel-cohereupdated.xlsx'
    },
]

for file in files:
    update_translations_with_zero_bleu(file['input_file'], file['output_file'])


Columns in the Excel file: Index(['English', 'Actual_Translation', 'Cohere_Translation', 'BLEU_Score'], dtype='object')
No rows with a BLEU score of 0 found.
Columns in the Excel file: Index(['English', 'Actual_Translation', 'Cohere_Translation', 'BLEU_Score'], dtype='object')
Regenerated: Argentina is well known for having one of the best polo teams and players in the world. -> আর্জেন্টিনা জানা মানে সে বেস্ট পোলো টিম এবং খেলোয়াড়দের থাকা, বিশ্বের সেরাদের মধ্যে একটি। | New BLEU Score: 0.024869542143125947
Regenerated: The village of Haldarsvík offer views of the nearby island Eysturoy and has an unusual octagonal church. -> হালদারসভিক গ্রামে থেকে পাওয়া যায় পরিচিত দ্বীপ এস্টুরোয়ের দৃশ্য এবং এখানে রয়েছে একটি অসাধারণ আটপাশের চর্চ। | New BLEU Score: 0.01755638228482643
Regenerated: However, the percentage of XDR-TB in the entire group of people with tuberculosis still seems to be low; 6,000 of the total 330,000 people infected at any particular moment in South Africa. -> সমগ্র তবেরকুল

In [58]:
import pandas as pd

def check_zero_bleu_scores(file_paths):
    for file_path in file_paths:
        print(f"\nChecking file: {file_path}")
        
        try:
            # Load the Excel file
            df = pd.read_excel(file_path)
            
            # Ensure the BLEU score column exists
            if 'BLEU_Score' not in df.columns:
                print("Error: 'BLEU_Score' column not found in the file.")
                continue

            # Find rows with a BLEU score of 0
            zero_bleu_rows = df[df['BLEU_Score'] == 0]
            
            if zero_bleu_rows.empty:
                print("No rows with a BLEU score of 0 found.")
            else:
                print(f"Rows with a BLEU score of 0: {len(zero_bleu_rows)}")
                print(zero_bleu_rows)
                
        except Exception as e:
            print(f"Error processing file {file_path}: {e}")

# Specify your file paths here
file_paths = [
    r'D:\Machine translation model\Machine Translation Model\OUTPUT EVALUATION\Cohere eval\evaluation_eng_ben-cohereupdated(1).xlsx',
    r'D:\Machine translation model\Machine Translation Model\OUTPUT EVALUATION\Cohere eval\evaluation_eng_guj-cohereupdated(1).xlsx',
    r'D:\Machine translation model\Machine Translation Model\OUTPUT EVALUATION\Cohere eval\evaluation_eng_mar-cohereupdated(1).xlsx',
    r'D:\Machine translation model\Machine Translation Model\OUTPUT EVALUATION\Cohere eval\evaluation_eng_tam-cohereupdated(1).xlsx',
    r'D:\Machine translation model\Machine Translation Model\OUTPUT EVALUATION\Cohere eval\evaluation_eng_tel-cohereupdated(1).xlsx'
]

check_zero_bleu_scores(file_paths)




Checking file: D:\Machine translation model\Machine Translation Model\OUTPUT EVALUATION\Cohere eval\evaluation_eng_ben-cohereupdated(1).xlsx
Error processing file D:\Machine translation model\Machine Translation Model\OUTPUT EVALUATION\Cohere eval\evaluation_eng_ben-cohereupdated(1).xlsx: [Errno 2] No such file or directory: 'D:\\Machine translation model\\Machine Translation Model\\OUTPUT EVALUATION\\Cohere eval\\evaluation_eng_ben-cohereupdated(1).xlsx'

Checking file: D:\Machine translation model\Machine Translation Model\OUTPUT EVALUATION\Cohere eval\evaluation_eng_guj-cohereupdated(1).xlsx
No rows with a BLEU score of 0 found.

Checking file: D:\Machine translation model\Machine Translation Model\OUTPUT EVALUATION\Cohere eval\evaluation_eng_mar-cohereupdated(1).xlsx
No rows with a BLEU score of 0 found.

Checking file: D:\Machine translation model\Machine Translation Model\OUTPUT EVALUATION\Cohere eval\evaluation_eng_tam-cohereupdated(1).xlsx
Rows with a BLEU score of 0: 1
      

In [70]:
import pandas as pd
import cohere
import random
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# Initialize Cohere client
cohere_api_key = 'API KEYS'
co = cohere.Client(cohere_api_key)

def regenerate_translation(english_sentence, target_language, previous_translation=None):
    if previous_translation:
        prompt = (
            f"Improve the following translation from English to **{target_language}**. "
            f"The previous translation was: '{previous_translation}'. "
            "Please ensure that the new translation is more fluent and accurate.\n\n"
            f"English sentence: {english_sentence}\n"
            f"New Translation in {target_language}:"
        )
    else:
        prompt = (
            f"Translate the following English sentence into **{target_language} only**, ensuring the translation is natural and fluent. "
            f"Please only use one of these languages: Tamil, Telugu. "
            f"Make sure that the translation is grammatically correct and contextually appropriate.\n\n"
            f"English sentence: {english_sentence}\n"
            f"Translation in {target_language}:"
        )

    try:
        response = co.generate(
            model='command-r-plus',
            prompt=prompt,
            max_tokens=500,
            temperature=random.uniform(0.3, 0.7)
        )
        return response.generations[0].text.strip()
    except Exception as e:
        print(f"Error in translation: {e}")
        return ""

def calculate_bleu_score(actual_translation, cohere_translation):
    smoothie = SmoothingFunction().method4
    return sentence_bleu([actual_translation.split()], cohere_translation.split(), smoothing_function=smoothie)

def update_translations_with_zero_bleu(input_file, output_file):
    # Load the Excel file
    df = pd.read_excel(input_file)

    print("Columns in the Excel file:", df.columns)

    # Ensure required columns exist
    required_columns = ['English', 'Actual_Translation', 'Cohere_Translation', 'BLEU_Score']
    for col in required_columns:
        if col not in df.columns:
            print(f"Error: Required column '{col}' not found in the input file.")
            return

    # Identify rows with a BLEU score of 0
    zero_bleu_rows = df[df['BLEU_Score'] == 0]

    if zero_bleu_rows.empty:
        print("No rows with a BLEU score of 0 found.")
        return

    target_language_map = {
        'evaluation_eng_tam-cohereupdated': 'Tamil',
        'evaluation_eng_tel-cohereupdated': 'Telugu'
    }
    target_language = None
    for key in target_language_map.keys():
        if key in input_file:
            target_language = target_language_map[key]
            break
    
    if target_language is None:
        print("Error: Unrecognized input file for setting target language.")
        return

    for index, row in zero_bleu_rows.iterrows():
        english_sentence = row['English']
        actual_translation = row['Actual_Translation']

        new_bleu_score = 0
        cohere_translation = ""
        attempts = 0
        previous_translation = None

        max_attempts = 5
        while new_bleu_score <= 0.1 and attempts < max_attempts:
            attempts += 1
            cohere_translation = regenerate_translation(english_sentence, target_language, previous_translation)

            if cohere_translation:
                new_bleu_score = calculate_bleu_score(actual_translation, cohere_translation)
                print(f"Attempt {attempts}: {english_sentence} -> {cohere_translation} | New BLEU Score: {new_bleu_score}")

                previous_translation = cohere_translation
            else:
                print(f"Failed to regenerate translation for: {english_sentence}")
                break

        if new_bleu_score > 0.1:
            df.at[index, 'Cohere_Translation'] = cohere_translation
            df.at[index, 'BLEU_Score'] = new_bleu_score
            print(f"Updated: {english_sentence} -> {cohere_translation} | Final BLEU Score: {new_bleu_score}")
        else:
            print(f"Max attempts reached for: {english_sentence}. Keeping previous translation.")

    df.to_excel(output_file, index=False)
    print(f"Updated data saved to: {output_file}")

# List of files to process
files_to_process = [
    (r'D:\Machine translation model\Machine Translation Model\OUTPUT EVALUATION\Cohere eval\evaluation_eng_tam-cohereupdated(1).xlsx',
     r'D:\Machine translation model\Machine Translation Model\OUTPUT EVALUATION\Cohere eval\evaluation_eng_tam-cohereupdatedW.xlsx'),
    (r'D:\Machine translation model\Machine Translation Model\OUTPUT EVALUATION\Cohere eval\evaluation_eng_tel-cohereupdated(1).xlsx',
     r'D:\Machine translation model\Machine Translation Model\OUTPUT EVALUATION\Cohere eval\evaluation_eng_tel-cohereupdatedW.xlsx'),
]

for input_file, output_file in files_to_process:
    update_translations_with_zero_bleu(input_file, output_file)

Columns in the Excel file: Index(['English', 'Actual_Translation', 'Cohere_Translation', 'BLEU_Score'], dtype='object')
Attempt 1: One of the main Christian tenets is that wealth should be used to alleviate suffering and poverty and that the monetary funds of the church are there specifically for that reason. -> ஒரு முக்கிய கிரிஸ்தவ நம்பிக்கை என்பது, செல்வாக்கு துன்பத்தையும் வறுமையையும் நீக்குவதற்காக பயன்படுத்தப்பட வேண்டும் என்பதும், அதற்காகவே தேவாலயத்தின் நிதி நிதி இருப்பதாகும். | New BLEU Score: 0
Attempt 2: One of the main Christian tenets is that wealth should be used to alleviate suffering and poverty and that the monetary funds of the church are there specifically for that reason. -> ஒரு முக்கிய கிறித்தவ நம்பிக்கை என்பது, செல்வாக்கு துன்பத்தையும் வறுமையையும் நீக்குவதற்காக பயன்படுத்தப்பட வேண்டும் என்பதும், அதற்காகவே தேவாலயத்தின் நிதி இருப்பதாகும். | New BLEU Score: 0
Attempt 3: One of the main Christian tenets is that wealth should be used to alleviate suffering and poverty and th